# Arbeitspaket (AP) 2: Retrieval Augmented Generation

### Persönliche Angaben (bitte ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td></td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>16.12.2024 / Raum: SF O3.54 / 8:00 – 12:30</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.24HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 48)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. grundlegende Fehler beim Einlesen der Daten). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. die Datentypen der eingelesenen Daten entsprechen nicht den Anforderungen gemäss Fragestellung). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. Spaltenname falsch geschrieben, TimeStamp nicht korrekt formatiert) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>



## Python Libraries und Settings

## <b>Vorbereitung (Dieser Teil wird <u>nicht</u> bewertet!)</b>

#### <b>1.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks von diesem Github Repository</b>
#### <b>2.) Führen Sie bitte die untenstehenden 2 Code Zellen aus sobald der Codespace gestartet ist und die libraries installiert hat</b>

In [67]:
import os  # For interacting with the operating system, e.g., file paths
import glob  # For working with file patterns
import asyncio  # For managing asynchronous tasks
import json  # For working with JSON data
import random  # For generating random data

from dotenv import load_dotenv  # For loading environment variables from a .env file
from PyPDF2 import PdfReader  # For reading PDF files
from tqdm import tqdm  # For displaying progress bars in loops
import pandas as pd  # For data manipulation and analysis

from langchain.text_splitter import RecursiveCharacterTextSplitter  # For splitting text into manageable chunks
from langchain.prompts import PromptTemplate  # For defining and managing prompt templates
from langchain_core.output_parsers import StrOutputParser  # For parsing string outputs from models
from langchain.chains.combine_documents import create_stuff_documents_chain  # For combining retrieved documents into a coherent chain
from langchain.globals import set_debug  # For enabling debug mode in LangChain

from langchain.text_splitter import RecursiveCharacterTextSplitter  # For splitting text into manageable chunks
from langchain.prompts import PromptTemplate  # For defining and managing prompt templates
from langchain.chains.combine_documents import create_stuff_documents_chain  # For combining retrieved documents into a coherent chain
from langchain.globals import set_debug  # For enabling debug mode in LangChain

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI  # For Google Generative AI integrations
from langchain_huggingface import HuggingFaceEmbeddings  # For generating embeddings using Hugging Face models
from langchain_openai import ChatOpenAI  # For OpenAI chat models
from langchain_groq import ChatGroq  # For Groq-based models

from langchain_community.vectorstores import FAISS, Chroma  # For storing and retrieving embeddings using FAISS and Chroma

import openai
from openai import OpenAI  # For OpenAI API access
from huggingface_hub import InferenceClient  # For Hugging Face Hub integrations

In [2]:
load_dotenv()
groq_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")
google_key = os.getenv("GOOGLE_API_KEY")


## <b>Aufgaben (Dieser Teil wird bewertet!)</b>
### Hinweise zum folgenden Arbeitspaket:

Im Rahmen dieses Arbeitspakets sollen Sie einen Retrieval-Augmented Generation (RAG)-Prozess erstellen, der medizinische Informationen aus Beipackzetteln gängiger Medikamente effizient abruft. Stellen Sie sich vor, Sie entwickeln ein System für Apotheker*innen oder medizinisches Personal, um schnell und präzise Fragen zu Medikamenten zu beantworten. Die folgenden fünf Beipackzettel stehen Ihnen als Datenquelle zur Verfügung:

- [data/Amoxicillin.pdf](data/Amoxicillin.pdf)
- [data/bisoprolol.pdf](data/bisoprolol.pdf)
- [data/citalopram.pdf](data/citalopram.pdf)
- [data/metformin.pdf](data/metformin.pdf)
- [data/paracetamol.pdf](data/paracetamol.pdf)

Ihre Aufgabe ist es, eine RAG-Pipeline zu implementieren, die relevante Informationen aus den Beipackzetteln abruft und in den Antwortprozess integriert. Nutzen Sie dabei die bereitgestellten Beschreibungen und Ihre Kenntnisse aus den Übungen.

### Erwartetes Ergebnis:

1. Lesen Sie die bereitgestellten Beipackzettel ein und extrahieren Sie den gesamten Text.
2. Zerlegen Sie den extrahierten Text in handhabbare Abschnitte mithilfe eines Text-Splitters (z. B. `RecursiveCharacterTextSplitter`).
3. Erstellen Sie Embeddings für die Textabschnitte mit einem vorgegebenen Modell.
4. Indexieren Sie die Embeddings in einem Vektorspeicher (z. B. FAISS).
5. Erarbeiten Sie einen passenden Prompt.
6. Bauen Sie die Chain zusammen,
7. Erstellen Sie eine Lsite mit Testfragen
8. Lassen Sie die 10 Fragen von ihrem RAG beantworten.

### Einreichungsdokumente:

Die Einreichung dieser Aufgabe sollte Folgendes umfassen:
- Das von Ihnen bearbeitete Notebook (dieses File).


<b style="color:blue;">Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul style="color:blue;">
  <li>Beachten Sie auch die zu jeder Aufgabenstellung zugehörenden Details zur Aufgabenstellung.</li>
  <li>Lösen Sie jede Aufgabe mit Hilfe von Python Code. Integrieren Sie den Python Code in die Code-Zellen der jeweiligen Aufgabe.</li>
  <li>Stellen Sie die Lösung(en) jeder Aufgabe so dar wie un der Aufgabe verlangt.</li>
  <li>Verwenden Sie als Unterstützung die Langchain Dokumentation, sowie die API Dokumentation der verwendeten Languge Models.</li>
</ul>

#### <b>Aufgabe (1): Lesen sie alle 5 PDFs aus dem Ordner data ein und speichern sie deren Inhalt als String ab</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die Files im Order 'data'.
- Speichern Sie die Inhalte aller 5 PDFs in einem einzigen String.
- Zeigen Sie mit Hilfe eines geeigneten Python Befehls die Dimension des Strings an (wie viele Zeichen?).
- Stellen Sie die ersten 100 Zeichen im Jupyter notebook dar.
<b style="color: gray;">(max. erreichbare Punkte: 6)</b>

In [3]:
# Einlesen der PDFs als String



In [1]:
# Anzeigen der Anzal Zeichen im String


# Anzeigen der ersten 100 Zeichen im String


#### <b>Aufgabe (2): Splitten Sie die Texte in Chunks von einer Länge von 1000 Zeichen. Geben sie ebenfalls einen overlap an.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie dazu den String, den sie in der vorherigen Aufgabe erstellt haben.
- Hinweis: In den Übungen haben wir den `RecursiveCharacterTextSplitter` verwendet.
- Stellen Sie die Menge aller Chunks im Jupyter Notebook dar.
- Stellen Sie die Länge des ersten Chunks im Jupyter Notenbook dar.

<b style="color: gray;">(max. erreichbare Punkte: 6)</b>

In [2]:
# Spliting in Chunks

In [3]:
# Darstellen der Menge aller Chunks

# Darstellen der Länge des ersten Chunks


#### <b>Aufgabe (3): Initialisieren sie ein Embedding Model</b>
<b>Details zur Aufgabenstellung:</b>
- Wählen sie ein geeignetes Embedding Modell von Huggingface aus.
- [Huggigface Modelle](https://huggingface.co/spaces/mteb/leaderboard).
- Beachten Sie die grösse des Modells. Das Modell sollte in ihrem Codespace ausführbar sein.
- Beachten Sie auch, dass unser Datensatz in deutscher Spracher verfasst ist.

<b style="color: gray;">(max. erreichbare Punkte: 2)</b>

In [7]:
# Initialisieren des Modells


#### <b>Aufgabe (4): Erstellen sie mit Hilfe der Chunks und Embeddings einen Vector Store und einen Retriever.</b>
<b>Details zur Aufgabenstellung:</b>
- Erstellen sie einen Vector Store
- Erstellen sie daraus einen Retriever
- Definieren sie selber die Anzahl Dokumente, welche der Retriever zurückgeben soll.
- Testen sie den Retriever mit der folgenden Anfrage: `"Welche Dosierung von Amoxicillin Axapharm wird für die Behandlung einer Endokarditis-Prophylaxe bei Erwachsenen empfohlen?"`
- Speichern Sie die Chunks in einer Variable, damit Sie diese wiederverwenden können.
- Falls die erhaltenen Chunks nicht relevant sind, erhöhen sie gegebenfalls die Anzahl Chunks die gesucht werden soll und führen Sie die Abfrage erneut durch. 
- Es muss nicht perfekt sein und falls sich nichts verbessert, fahren Sie weiter mit dem aktuellen Ergebnis.
<b style="color: gray;">(max. erreichbare Punkte: 6)</b>

In [5]:
# Erstellen des Vector Stores

# Erstellen des Retrievers




In [ ]:
# Testen des retrievers
docs = ...
print(docs)

#### <b>Aufgabe (5): Prompt Engineering.</b>
<b>Details zur Aufgabenstellung:</b>
- Erstellen Sie ein an die Situation angepasstes PromptTemplate, in welches die erhaltenen Chunks und eine Frage eingefügt werden können.
- Das PromptTemplate soll zwei`input_variables` enthalten. Eine für die Frage und eine für die Dokumente.
- Inspiration zum Prompt Engineering: Es richtet sich an medizisches Fachpersonal, gegebenfalls Antwortlänge, was soll mit den eingefügten Texten passierent.
- Beachten Sie die Sprache der Texte.
<br><br>


<b style="color: gray;">(max. erreichbare Punkte: 6)</b>

In [6]:
# Erstellen sie einen Prompt

# Erzeugen sie daraus ein Prompt Template


#### <b>Aufgabe (6): Setzen sie die Chain zusammen</b>
<b>Details zur Aufgabenstellung:</b>
- Kombinieren Sie die bisher erstellten Komponenten in einer Retrieval-Augmented-Generation (RAG)-Chain.
- Die Chain sollte mindestens folgende Schritte enthalten:
  - Einen Retriever, der relevante Textchunks basierend auf der Eingabefrage abruft.
  - Ein Language Model (LLM), das basierend auf den abgerufenen Chunks eine Antwort generiert.
  - einen Outputparser
- Testen Sie Ihre Chain mit folgender Frage: 
`Welche Dosierung von Amoxicillin Axapharm wird für die Behandlung einer Endokarditis-Prophylaxe bei Erwachsenen empfohlen?`
<br><br>

<b style="color: gray;">(max. erreichbare Punkte: 6)</b>

In [8]:
# set language model and output parser



In [7]:
# define complete chain (3 parts)


In [9]:
# Test query
query = "Welche Dosierung von Amoxicillin Axapharm wird für die Behandlung einer Endokarditis-Prophylaxe bei Erwachsenen empfohlen?"

In [10]:
# print result of test query with your chain (hint: input is a dictionary)


### <b>Aufgabe (7): Reusable Function für die RAG-Pipeline</b>
<b>Details zur Aufgabenstellung:</b>
- Schreiben Sie eine Funktion `get_answer_and_documents`, die eine Frage mit Hilfe der RAG-Pipeline beantwortet.
- Die Funktion sollte:
  - Einen Parameter für die Frage (`question`), die RAG-Chain (`rag_chain`), und den Retriever (`retriever`) enthalten.
  - Dokumente mit dem Retriever abrufen und als Kontext an die Chain übergeben.
  - Die Antwort und die verwendeten Dokumente zurückgeben.
- Testen Sie die Funktion mit der Frage: `Welche Dosierung von Amoxicillin Axapharm wird für die Behandlung einer Endokarditis-Prophylaxe bei Erwachsenen empfohlen?`

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [ ]:
# function get_answer_and_documents
def get_answer_and_documents(..., ..., ...,):
   
    return reply, retrieved_documents

In [69]:
query2 = "Ab welcher Kreatinin-Clearance ist die Einnahme von Metformin kontraindiziert?"

In [ ]:
# test your function with query2
answer, documents = get_answer_and_documents()

In [ ]:
# this should now print the answer and the documents, as you return a touple of answer and documents
print("retrieved answer: " + answer)
print("retrieved documents: " + str(documents))

Als Untersützung: So erhalten sie die Antwort au

#### <b>Aufgabe (7): Erstellen sie eine Liste mit Testfragen.</b>
<b>Details zur Aufgabenstellung:</b>
- Erstellen sie eine Python Liste mit 10 Fragen, zu diesen Medikatementen.
- Sie Können die Fragen mit einem Sprachmodell generieren lassen und in einer Liste abspeichern.
- Dazu brauchen sie einen geegineten Prompt und eine Schleife um mehrere Anfragen an das Modell zu senden.
- Notfalls kann die Liste von Hand erstellt werden oder über mehrere Zeilen Hinweg.
- Sie können dazu Chunks aus den Packungsbeilagen verwenden, müssen aber nicht.
- geben sie am Ende die Liste ihrer Fragen aus

<b style="color: gray;">(max. erreichbare Punkte: 6)</b>

In [18]:
# Generieren der Testfragen (Hinweis: Es kann hilfreich sein, die Fragen in einer Liste zu speichern)
# Sie dürfen auch Hilfslisten verwenden, wie zum Beispiel die Namen der Medikamente, oder ähnliches.

In [19]:
# Augabe aller Test Fragen

#### <b>Aufgabe (8): Lassen sie 10 die Fragen von ihrem Retriever Beantworten.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden sie die 10 generierten Fragen und lassen sie diese von ihrer Chain beantworten
- Geben Sie dazu Dokumente und Antwort aus.
- Geben sie selbst eine Einschätzung ab, ob ihre Chain funktionert oder nicht.
- Geben sie dazu ein Beispiel was gut ging und was nicht.

<b style="color: gray;">(max. erreichbare Punkte: 6</b>

In [20]:
# Beantwortung der 10 generierten Fragen


#### Ihre Einschätzung zur Qualität (mit Doppelklick auf die Zelle können sie reinschreiben):

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')